# **Brightway2 Basics**

This Jupyter Notebook is intended to outline the basic usage of Brightway2 package. First, we import all necessary packages, libraries and databases. 

In [1]:
from brightway2 import *

In [2]:
import bw2io 

In [3]:
from bw2io.strategies import *

In [4]:
import os
import matplotlib.pyplot as plt

In [5]:
import numpy as np

In [6]:
import pandas as pd

In [7]:
# List the current BW2 projects
list(projects) 

[Project: default,
 Project: BW2 Introduction,
 Project: CPACS Turbofan,
 Project: TF_Hydrogen,
 Project: Test_1,
 Project: test,
 Project: fuselage,
 Project: ATA Chapters,
 Project: alicia,
 Project: Jupyter_Lab,
 Project: bw2-project,
 Project: CPACS TF Test]

In [8]:
projects.set_current("CPACS Turbofan")
bw2io.bw2setup()

Biosphere database already present!!! No setup is needed


# **Importing the ecoinvent database**

Here, we want to import the ecoinvent database so that it is always available.

In [9]:
list(databases)

['biosphere3', 'ecoinvent_3.9.1_cutoff', 'production database', 'D250-TF-2040']

In [10]:
path391 = (r"D:\ecoinvent391\datasets")
ei391 = SingleOutputEcospold2Importer(path391, 'ecoinvent_3.9.1_cutoff')
ei391.apply_strategies()
ei391.statistics()

Extracting XML data from 21238 datasets
Extracted 21238 datasets in 138.53 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy:

(21238, 674593, 0)

In [12]:
ei391.write_database()
ei391

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:53


Title: Writing activities to SQLite3 database:
  Started: 06/30/2023 10:53:58
  Finished: 06/30/2023 10:54:52
  Total time elapsed: 00:00:53
  CPU %: 98.70
  Memory %: 10.88
Created database: ecoinvent_3.9.1_cutoff


In [13]:
if 'ecoinvent_3.9.1_cutoff' in databases:
    print("ecoinvent database has already been imported")

else:    
    path391 = (r"D:\ecoinvent391\datasets")
    ei391 = SingleOutputEcospold2Importer(path391, 'ecoinvent_3.9.1_cutoff')
    ei391.apply_strategies()
    ei391.statistics()
    ei391.write_database()

ecoinvent database has already been imported


In [14]:
eidb = Database('ecoinvent 3.9.1_cutoff')

In [15]:
list(databases)

['biosphere3', 'ecoinvent_3.9.1_cutoff', 'production database', 'D250-TF-2040']

# **Importing Excel databases - Life Cycle Inventories (LCIs)**
There are two data types when looking at databases inside BW2: Excel and bw2package. In our case, it is interesting to use parameters for easy data update. Since bw2package does not support importing parameters, we'll have to choose for Excel data import.

At first, we import the database with the different materials used for the aircraft production. We use *ExcelImporter* to import the db excel sheet. Then, we need to match the databases considering name, unit and location with the ecoinvent db.

In [16]:
materials = ExcelImporter("C:\jupyter_notebooks\materials.xlsx")
materials.apply_strategies()
materials.match_database(fields=('name', 'unit', 'location'))
materials.match_database("ecoinvent_3.9.1_cutoff", fields=('name', 'unit', 'location','reference product'))
materials.statistics()
list(databases)
materials.write_excel()
materials.write_database()

Extracted 1 worksheets in 0.05 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 19.56 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


13 datasets
60 exchanges
0 unlinked exchanges
  
Wrote matching file to:
C:\Users\alba_jo\AppData\Local\pylca\Brightway3\CPACS-Turbofan.ed96b4406da7214d1081f27731aa743d\output\db-matching-production-database.xlsx


0% [#############] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 06/30/2023 10:57:42
  Finished: 06/30/2023 10:57:42
  Total time elapsed: 00:00:00
  CPU %: 104.20
  Memory %: 11.73
Created database: production database


Then, we import the aircraft database (in this case, for the Conventional Turbofan layout) and match the respective databases: ecoinvent and production database. 

In [17]:
TF = ExcelImporter("C:\jupyter_notebooks\D250-TF-2040.xlsx")
TF.apply_strategies()
TF.match_database(fields=('name', 'unit', 'location'))
TF.match_database("ecoinvent_3.9.1_cutoff", fields=('name', 'unit', 'location', 'reference product'))
TF.match_database("production database", fields=('name', 'unit', 'location','reference product'))
TF.statistics()
list(databases)
TF.write_database()

Extracted 1 worksheets in 0.05 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 19.09 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields


Writing activities to SQLite3 database:


Applying strategy: link_iterable_by_fields
36 datasets
98 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 06/30/2023 10:58:06
  Finished: 06/30/2023 10:58:06
  Total time elapsed: 00:00:00
  CPU %: 195.30
  Memory %: 11.33
Created database: D250-TF-2040


In [18]:
for unlinked in TF.unlinked: # checking if there are any unlinked exchanges and listing them
    print(unlinked)

# **LCIA methods**

The following LCIA methods can be used within BW2. The chosen method is Environmental Footprint 3.0 (EF 3.0).  

In [19]:
methods

Methods dictionary with 762 objects, including:
	('CML v4.8 2016', 'acidification', 'acidification (incl. fate, average Europe total, A&B)')
	('CML v4.8 2016', 'climate change', 'global warming potential (GWP100)')
	('CML v4.8 2016', 'ecotoxicity: freshwater', 'freshwater aquatic ecotoxicity (FAETP inf)')
	('CML v4.8 2016', 'ecotoxicity: marine', 'marine aquatic ecotoxicity (MAETP inf)')
	('CML v4.8 2016', 'ecotoxicity: terrestrial', 'terrestrial ecotoxicity (TETP inf)')
	('CML v4.8 2016', 'energy resources: non-renewable', 'abiotic depletion potential (ADP): fossil fuels')
	('CML v4.8 2016', 'eutrophication', 'eutrophication (fate not incl.)')
	('CML v4.8 2016', 'human toxicity', 'human toxicity (HTP inf)')
	('CML v4.8 2016', 'material resources: metals/minerals', 'abiotic depletion potential (ADP): elements (ultimate reserves)')
	('CML v4.8 2016', 'ozone depletion', 'ozone layer depletion (ODP steady state)')
Use `list(this object)` to get the complete list.

In [20]:
create_default_lcia_methods(overwrite=True)

Wrote 762 LCIA methods with 227223 characterization factors


In [21]:
EF30 = [method for method in methods if 'EF v3.0 no LT' in str(method)]
print(type(EF30))
EF30
len(EF30)

<class 'list'>


28

After choosing the LCIA method, we can then proceed to the actual LCA calculation.

# **LCA Calculation**
In this step, we start combining the materials and aircraft production databases, as well as the respective ecoinvent activities. 

In [34]:
ei391 = Database('ecoinvent_3.9.1_cutoff')
ei391

Brightway2 SQLiteBackend: ecoinvent_3.9.1_cutoff

In [35]:
materials_db = Database('production database')
materials_db

Brightway2 SQLiteBackend: production database

In [76]:
TF_db = Database('D250-TF-2040')
TF_db

Brightway2 SQLiteBackend: D250-TF-2040

In [77]:
materials_check = [activity for activity in materials_db if "Steel" in activity['name']]
materials_check

['High-Strength Steel' (kilogram, GLO, None)]

In [135]:
aircraft_check = [activity for activity in TF_db if "ATA-" in activity['name']]
aircraft_check


['ATA-02_Operations_Catering' (unit, GLO, None),
 'ATA-55_Stabilizers_VTP' (unit, GLO, None),
 'ATA-39_Electrical/Electronic_Seat Electronic System' (unit, GLO, None),
 'ATA-55_Stabilizers_HTP' (unit, GLO, None),
 'ATA-72_Engine_fan_prop_Fan' (unit, GLO, None),
 'ATA-72_Engine_fan_prop_Gear Box' (unit, GLO, None),
 'ATA-57_wings_Fixed Leading Edge' (unit, GLO, None),
 'ATA-32_Landing-gear_Landing Gear Supports' (unit, GLO, None),
 'ATA-32_Landing-gear_Landing Gears' (unit, GLO, None),
 'ATA-72_Engine_fan_prop_Propeller' (unit, GLO, None),
 'ATA-02_Operations_Crew' (unit, GLO, None),
 'ATA-25_equipment-furnishing_Crew Rest' (unit, GLO, None),
 'ATA-02_Operations_Operator Items' (unit, GLO, None),
 'ATA-57_wings_Movable Leading Edge' (unit, GLO, None),
 'ATA-57_wings_Wing Miscellaneous' (unit, GLO, None),
 'ATA-38_water-waste_Water' (unit, GLO, None),
 'ATA-79_Oil_Oil' (unit, GLO, None),
 'ATA-28_Fuel_Unusable Fuel' (unit, GLO, None),
 'ATA-54_Nacelles/Pylons_Pylon' (unit, GLO, None),
 '

In [87]:
ATA_02 = [activity for activity in TF_db if "ATA-02" in activity['name']]
ATA_02

['ATA-02_Operations_Aircraft Documents' (unit, GLO, None),
 'ATA-02_Operations_Catering' (unit, GLO, None),
 'ATA-02_Operations_Crew' (unit, GLO, None),
 'ATA-02_Operations_Operator Items' (unit, GLO, None)]

In [88]:
ATA_25 = [activity for activity in TF_db if "ATA-25" in activity['name']]
ATA_25

['ATA-25_equipment-furnishing_Emergency Equipment' (unit, GLO, None),
 'ATA-25_equipment-furnishing_Galley Structure' (unit, GLO, None),
 'ATA-25_equipment-furnishing_Furnishing' (unit, GLO, None),
 'ATA-25_equipment-furnishing_Crew Rest' (unit, GLO, None),
 'ATA-25_equipment-furnishing_Passenger Seats' (unit, GLO, None)]

In [90]:
ATA_28 = [activity for activity in TF_db if "ATA-28" in activity['name']]
ATA_28

['ATA-28_Fuel_Unusable Fuel' (unit, GLO, None)]

In [91]:
ATA_32 = [activity for activity in TF_db if "ATA-32" in activity['name']]
ATA_32

['ATA-32_Landing-gear_Landing Gears' (unit, GLO, None),
 'ATA-32_Landing-gear_Landing Gear Supports' (unit, GLO, None)]

In [94]:
ATA_38 = [activity for activity in TF_db if "ATA-38" in activity['name']]
ATA_38

['ATA-38_water-waste_Water' (unit, GLO, None),
 'ATA-38_water-waste_Fluid Toilets' (unit, GLO, None)]

In [93]:
ATA_39 = [activity for activity in TF_db if "ATA-39" in activity['name']]
ATA_39

['ATA-39_Electrical/Electronic_Seat Electronic System' (unit, GLO, None)]

In [92]:
ATA_54 = [activity for activity in TF_db if "ATA-54" in activity['name']]
ATA_54

['ATA-54_Nacelles/Pylons_Pylon' (unit, GLO, None),
 'ATA-54_Nacelles/Pylons_Nacelle' (unit, GLO, None)]

In [95]:
ATA_55 = [activity for activity in TF_db if "ATA-55" in activity['name']]
ATA_55

['ATA-55_Stabilizers_VTP' (unit, GLO, None),
 'ATA-55_Stabilizers_HTP' (unit, GLO, None)]

In [96]:
ATA_57 = [activity for activity in TF_db if "ATA-57" in activity['name']]
ATA_57

['ATA-57_wings_Foldable Wing Mechanism' (unit, GLO, None),
 'ATA-57_wings_Fixed Leading Edge' (unit, GLO, None),
 'ATA-57_wings_Wing Structure' (unit, GLO, None),
 'ATA-57_wings_Fixed Trailing Edge' (unit, GLO, None),
 'ATA-57_wings_Wing Tips' (unit, GLO, None),
 'ATA-57_wings_Wing Miscellaneous' (unit, GLO, None),
 'ATA-57_wings_Movable Leading Edge' (unit, GLO, None),
 'ATA-57_wings_Movable Trailing Edge' (unit, GLO, None)]

In [98]:
ATA_72 = [activity for activity in TF_db if "ATA-72" in activity['name']]
ATA_72

['ATA-72_Engine_fan_prop_Propeller' (unit, GLO, None),
 'ATA-72_Engine_fan_prop_Engine Systems' (unit, GLO, None),
 'ATA-72_Engine_fan_prop_Gear Box' (unit, GLO, None),
 'ATA-72_Engine_fan_prop_Gas Turbine' (unit, GLO, None),
 'ATA-72_Engine_fan_prop_Engine TF' (unit, GLO, None),
 'ATA-72_Engine_fan_prop_Fan' (unit, GLO, None)]

In [99]:
ATA_79 = [activity for activity in TF_db if "ATA-79" in activity['name']]
ATA_79

['ATA-79_Oil_Oil' (unit, GLO, None)]

In [137]:
act_df = pd.DataFrame()
df = pd.DataFrame()
act_df['Database Process'] = aircraft_check
act_df['Amount'] = 1
act_df.reset_index()
print(act_df)

                                     Database Process  Amount
0   [code, location, reference product, type, unit...       1
1   [code, location, reference product, type, unit...       1
2   [code, location, reference product, type, unit...       1
3   [code, location, reference product, type, unit...       1
4   [code, location, reference product, type, unit...       1
5   [code, location, reference product, type, unit...       1
6   [code, location, reference product, type, unit...       1
7   [code, location, reference product, type, unit...       1
8   [code, location, reference product, type, unit...       1
9   [code, location, reference product, type, unit...       1
10  [code, location, reference product, type, unit...       1
11  [code, location, reference product, type, unit...       1
12  [code, location, reference product, type, unit...       1
13  [code, location, reference product, type, unit...       1
14  [code, location, reference product, type, unit...       1
15  [cod

In [138]:
for index in act_df.index:
    lca = LCA({act_df['Database Process'][index]: act_df['Amount'][index]})
    lca.lci()
    for method in EF30:
        lca.switch_method(method)
        lca.lcia()
        impact_cat = str(method)
        impact_unit = str(methods[method]['unit'])
        header = impact_cat,'({})'.format(impact_unit)
        #act_df.loc[index, " ".join(header)] = lca.score        
        act_df.loc[index, " ".join(method[1:])] = lca.score
act_df = act_df.drop(["Database Process", "Amount"], axis=1)
    
act_df


,acidification no LT accumulated exceedance (AE) no LT,climate change no LT global warming potential (GWP100) no LT,climate change: biogenic no LT global warming potential (GWP100) no LT,climate change: fossil no LT global warming potential (GWP100) no LT,climate change: land use and land use change no LT global warming potential (GWP100) no LT,ecotoxicity: freshwater no LT comparative toxic unit for ecosystems (CTUe) no LT,"ecotoxicity: freshwater, inorganics no LT comparative toxic unit for ecosystems (CTUe) no LT","ecotoxicity: freshwater, metals no LT comparative toxic unit for ecosystems (CTUe) no LT","ecotoxicity: freshwater, organics no LT comparative toxic unit for ecosystems (CTUe) no LT",energy resources: non-renewable no LT abiotic depletion potential (ADP): fossil fuels no LT,...,"human toxicity: non-carcinogenic, inorganics no LT comparative toxic unit for human (CTUh) no LT","human toxicity: non-carcinogenic, metals no LT comparative toxic unit for human (CTUh) no LT","human toxicity: non-carcinogenic, organics no LT comparative toxic unit for human (CTUh) no LT",ionising radiation: human health no LT human exposure efficiency relative to u235 no LT,land use no LT soil quality index no LT,material resources: metals/minerals no LT abiotic depletion potential (ADP): elements (ultimate reserves) no LT,ozone depletion no LT ozone depletion potential (ODP) no LT,particulate matter formation no LT impact on human health no LT,photochemical oxidant formation: human health no LT tropospheric ozone concentration increase no LT,water use no LT user deprivation potential (deprivation-weighted water consumption) no LT
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
1,163.697498,29335.104596,55.164447,29245.399966,34.540183,6.997098e+05,5.178121e+04,6.423743e+05,5554.308630,3.456195e+05,...,4.491181e-05,2.677817e-04,1.022090e-05,838.066664,9.828817e+04,8.671355e-02,1.617313e-04,2.182140e-03,84.212312,6392.615133
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
4,147.487719,22703.293868,41.680085,22583.477654,78.136129,6.695621e+05,6.604669e+04,5.973977e+05,6117.679177,2.484177e+05,...,4.521075e-04,3.619629e-04,1.046399e-05,567.895497,2.208562e+05,1.612787e-01,1.565660e-03,2.401623e-03,96.959035,8726.539916
5,10.523962,1896.346830,3.479951,1890.893007,1.973873,6.373957e+04,4.025372e+03,5.931063e+04,403.574145,2.064716e+04,...,7.521724e-06,4.342280e-05,7.333312e-07,45.269093,9.970495e+03,4.699966e-02,1.836691e-05,1.486851e-04,6.789831,489.998620
6,174.945571,31350.794619,58.954937,31254.926156,36.913527,7.477887e+05,5.533923e+04,6.865135e+05,5935.959374,3.693679e+05,...,4.799781e-05,2.861817e-04,1.092320e-05,895.652366,1.050418e+05,9.267186e-02,1.728442e-04,2.332080e-03,89.998755,6831.868060
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
8,76.321561,13752.629938,25.237195,13713.077884,14.314860,4.622502e+05,2.919268e+04,4.301308e+05,2926.788383,1.497367e+05,...,5.454882e-05,3.149095e-04,5.318243e-06,328.299170,7.230773e+04,3.408495e-01,1.332000e-04,1.078290e-03,49.241013,3553.553379
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0

In [29]:
act_df.to_excel("TF.xlsx")

In [ ]:
act.plot.bar()